In [ ]:
# 一、选股,股池：
# (聚宽上我还没实现股池管理)
# 1、每日全市场选股入股池:
# (1)剔除st,退市、前一交易日停牌的股
# (2)选股条件：
# 连续三日缩量，当日量小于五日十日十五日均量
# (3)若标的已在股池中，选股日期设置为最新日期
# (4)每日清除在股池中超过5个交易日的标的
# 二、买入：
# 1、股池中符合买入条件:
# (1)当日量大于前一日量
# (2)当日量大于五日十日十五日均量
# (3)当日收盘价大于开盘价
# (4)当日KDJ金叉
# (5)买入前以换手及五日平均换手排序，换手大者优先买入(聚宽上我还没实现)
# (6)大盘23个交易日内出现macd死叉不买(刚想到，聚宽上我还没实现)


In [ ]:
import pandas as pd
import numpy as np

In [ ]:

def FILTERx(COND, N):

    k1 = pd.Series(np.where(COND, 1, 0), index=COND.index)
    idx = k1[k1 == 1].index.codes[0]
    needfilter = pd.Series(idx, index=idx)
    afterfilter = needfilter.diff().apply(lambda x: False if x > N else True)
    k1.iloc[afterfilter[afterfilter].index] = 2
    return k1.apply(lambda x: 1 if x == 2 else 0)

In [ ]:
def select1(data):
    # 连续三日缩量
    ch= data.close[-1] * 1.1
    cl= data.close[-1] * 0.9
#     ch= data.close * 1.1
#     cl = data.close * 0.9
    
    df=pd.concat([QA.MA(data.close, x) for x in (5,10,20,30,60,90,120,250) ], axis = 1).dropna()[-1:]
    df.columns = [u'm5',u'm10',u'm20',u'm30',u'm60',u'm90',u'm120', u'm250']  
    df_h = df.apply(lambda x:x.max() <= ch,  axis = 1 )
    df_l = df.apply(lambda x:x.min() >= cl,  axis = 1 )
    
    df['dfh'] = df_h
    df['dfl'] = df_l
#     out=df.iloc[-1].apply(lambda x: True if x>cl and x < ch else False)

    return df
 
#     return pd.DataFrame({
# #         't1': t1,
# #         't2': t2,
# #         't3': t3,
# #         'm5': MA_N[0],
# #         'm10':  MA_N[1],
# #         'm20':  MA_N[2],
# #         'm30':  MA_N[3],
# #         'm60':  MA_N[4],
# #         'm90':  MA_N[5],
# #         'm120':  MA_N[6],
# #         'm250':  MA_N[7],
#         'out':out,
# #         'man':MA_N
#     })
    


In [ ]:
#QA.QA_fetch_stock_block_adv().get_block('上证50').code

In [ ]:
import QUANTAXIS as QA

In [ ]:
# stock_list = QA.QA_fetch_stock_block_adv().get_block('中小300').code

In [32]:
stock_list = QA.QA_fetch_stock_list_adv()

In [33]:
stocklist_all = stock_list[~stock_list.name.apply(lambda x: 'ALL-ST' in x)].code.tolist()

In [34]:
# stocklist_all=stocklist_all[1:5]

In [35]:
# stocklist_all

['000002', '000004', '000005', '000006']

In [36]:
#blockname=QA.QA_fetch_stock_block_adv().block_name

In [37]:
data = QA.QA_fetch_stock_day_adv(stocklist_all,'2000-01-01','2020-11-30')

In [ ]:
data.selects('000002','2018-06-10').data

In [ ]:
# = QA.QA_fetch_stock_day_adv('000001','2019-07-01','2019-09-30')

In [ ]:
#data.selects('600532','2008-05-28','2008-06-10').data

In [41]:
ind = data.add_func(select1)
ind.query('dfh==True and dfl==True')

,,m5,m10,m20,m30,m60,m90,m120,m250,dfh,dfl
date,code,,,,,,,,,,
2020-04-07,000002,26.6,26.321,27.116,28.316333,29.057667,29.175111,28.573167,28.4054,True,True


In [40]:
ind.query('dfh==True and dfl==True')

,,m5,m10,m20,m30,m60,m90,m120,m250,dfh,dfl
date,code,,,,,,,,,,
2020-04-07,000002,26.6,26.321,27.116,28.316333,29.057667,29.175111,28.573167,28.4054,True,True


In [ ]:
x1 = ind.query('res==True').loc[:,['x1','x2','x3','x4']].all(axis=1).groupby(level=1).shift()


In [ ]:
x1[x1==True]

In [ ]:
ind.loc['2008-04-22':'2008-06-19']